In [ ]:
from datetime import datetime
import urllib.request
import base64
import json
import time
import os

webui_server_url = 'http://127.0.0.1:7860' # Runpod server URL 
base_out_dir = "/workspace/outputs/API-outputs/"  # Base output directory
os.makedirs(base_out_dir, exist_ok=True)

# Function to find the next available batch number
def find_next_batch_number(base_dir):
    existing_batches = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d)) and d.startswith('batch_')]
    if not existing_batches:
        return 1
    highest_batch_number = max([int(d.split('_')[1]) for d in existing_batches])
    return highest_batch_number + 1

def timestamp():
    return datetime.fromtimestamp(time.time()).strftime("%Y%m%d-%H%M%S")

def encode_file_to_base64(path):
    with open(path, 'rb') as file:
        return base64.b64encode(file.read()).decode('utf-8')

def decode_and_save_base64(base64_str, save_path, init_image_path, counter):
    with open(save_path, "wb") as file:
        file.write(base64.b64decode(base64_str))
    # Save the settings in a JSON file
    json_filename = f"{os.path.splitext(save_path)[0]}.json"
    with open(json_filename, 'w') as json_file:
        json.dump({"init_image_path": init_image_path, "counter": counter}, json_file, indent=4)

    print(f"Saved Image: {save_path} and Settings: {json_filename}")  # Print statement for saving files


def call_api(api_endpoint, **payload):
    data = json.dumps(payload).encode('utf-8')
    request = urllib.request.Request(
        f'{webui_server_url}/{api_endpoint}',
        headers={'Content-Type': 'application/json'},
        data=data,
    )
    response = urllib.request.urlopen(request)
    return json.loads(response.read().decode('utf-8'))

def call_img2img_api(init_image_path, batch_number, counter, **payload):
    payload['init_images'] = [encode_file_to_base64(init_image_path)]
    response = call_api('sdapi/v1/img2img', **payload)
    for index, image in enumerate(response.get('images', [])):
        batch_dir = os.path.join(base_out_dir, f'batch_{batch_number}')
        os.makedirs(batch_dir, exist_ok=True)
        save_path = os.path.join(batch_dir, f'{counter}.png')
        decode_and_save_base64(image, save_path, init_image_path, counter)
        counter += 1
    return counter

if __name__ == '__main__':
    image_directory = "/workspace/source/neon2/"  # Directory with initial images
    image_paths = sorted([os.path.join(image_directory, filename) for filename in os.listdir(image_directory) if filename.endswith(('.png', '.jpg'))], key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))

    batch_number = find_next_batch_number(base_out_dir)
    counter = 1
    for path in image_paths:
        print(f"Processing image: {path}")

        payload = {
            "prompt": """Visualize <lora:woodfigurez-sdxl:0.2>""",
       
            "negative_prompt": """cropped, watermark, plastic, metallic, logo, people, multiple flowers, flowers,
            woman, man, text, signature, copyright, writing, letters, human, woman, boobs, body, 
            fit, sexy,nsfw, angry, low quality, artefacts, bad art, poorly drawn, lowres, 
            simple, pixelated, grain, noise, blurry, wings, butterfly, cartoon, computer game, video game, painting, drawing, sketch,
            disfigured, deformed, ugly""",
        
            "sampler_name": "Euler",
       
            "batch_size": 1,  # Processing one image at a time
            "Batch_count":2,
            "steps": 36,
            "cfg_scale": 9,
            "width": 1024,
            "height": 1024,
            "denoising_strength": 0.64,
            "model_hash": "c83491d2fa",
            "model": "protovisionXLHighFidelity3D_release0630Bakedvae",
            "seed": -1,
       
        }

        
        counter = call_img2img_api(path, batch_number, counter, **payload)
    print(f"Completed Batch {batch_number}")  # Print statement for batch completion

    batch_number += 1
